In [ ]:
!pip install rlcard[torch]

In [ ]:
import rlcard
from rlcard.agents import DQNAgent
from rlcard.agents import RandomAgent
import os
import torch

게임 환경 생성

In [ ]:
env = rlcard.make("custom")
print("Number of actions:", env.num_actions)
print("Number of players:", env.num_players)
print("Shape of state:", env.state_shape)
print("Shape of action:", env.action_shape)

Number of actions: 473
Number of players: 2
Shape of state: [[6, 7, 4], [6, 7, 4]]
Shape of action: [None, None]


DQN agent 생성, 랜덤 agent 생성

In [ ]:
agent = DQNAgent(
    num_actions=env.num_actions,
    state_shape=env.state_shape[0],
    mlp_layers=[512, 512],
    learning_rate = 0.00005,
)
agents = [agent]
for _ in range(1, env.num_players):
        agents.append(RandomAgent(num_actions=env.num_actions))

저장된 DQN agent 불러올때 사용

In [1]:
# checkpoint = torch.load('saved_models/SavedDQNAgent.pth')
# agent = DQNAgent.from_checkpoint(checkpoint)
# agents = [agent]
# for _ in range(1, env.num_players):
#         agents.append(RandomAgent(num_actions=env.num_actions))

게임에 agent 둘 추가

In [ ]:
env.set_agents(agents)

In [ ]:
from rlcard.utils import (
    tournament,
    reorganize,
    Logger,
    plot_curve,
)

학습 시작

In [ ]:
with Logger("experiments/custom_dqn_result/") as logger:
    for episode in range(1000):

        # Generate data from the environment
        trajectories, payoffs = env.run(is_training=True)

        # Reorganaize the data to be state, action, reward, next_state, done
        trajectories = reorganize(trajectories, payoffs)

        # Feed transitions into agent memory, and train the agent
        for ts in trajectories[0]:
            agent.feed(ts)

        # Evaluate the performance.
        if episode % 50 == 0:
            logger.log_performance(
                episode,
                tournament(
                    env,
                    100,
                )[0]
            )

    # save model
    os.makedirs('saved_models', exist_ok=True)
    agent.save_checkpoint(path='saved_models', filename='SavedDQNAgent.pth')

    # Get the paths
    csv_path, fig_path = logger.csv_path, logger.fig_path

결과 학습 곡선

In [ ]:
plot_curve(csv_path, fig_path, "DQN")